In [1]:
import pandas as pd
import numpy as np
from scipy.optimize import minimize

In [2]:
data = pd.read_excel(r"C:\Users\zachy\Desktop\Gilberto\Book2.xlsx", index_col=0)
data = data.fillna(0)  # Read data, fill nan value

#### cov is the covariance matrix

In [3]:
cov = pd.read_excel(r"C:\Users\zachy\Desktop\Gilberto\Cov_matrix.xlsx", index_col=0)

for i in range(cov.shape[0]):
    cov.iloc[i,:] = cov.iloc[:,i]

#### define all the parameters

In [4]:
RSF = np.array([0.5, 0.65, 0, 0.15])
HQLA = np.array([0.25, 0.25, 1, 0.15])
RWA = np.array([1, 0.35, 0, 0.20])
ASF = np.array([0.5, 0.9, 0.95, 1, 1])

T = 360  # time period
DRE = 0.18
Eta = 0.63
Total_Bal = 1500000
NSFR_ratio = 1.1
LCR_ratio = 1.2
TLAC_ratio = 0.18

Haircut = 0.02
borrow_rate = 0.0017  #daily

In [5]:
a1_rate = data['a1 rate'].values
a2_rate = data['a2 rate'].values
a3_rate = data['a3 rate'].values
a4_rate = data['a4 rate'].values
l1_rate = data['l1 rate'].values
l2_rate = data['l2 rate'].values
l3_rate = data['l3 rate'].values
l4_rate = data['l4 rate'].values
l5_rate = data['l5 rate'].values

####  Our objective is to minimize the function 

#### $\sqrt[]{x^T\Sigma x}-x^T\mu$ 

#### where $\Sigma$ is the covariance matrix, $\mu$ is the returns and $x$ is the weight vector

In [6]:
def obj_mean_var(x):  # Defn Objective function
    
    returns = pd.DataFrame({'A1':a1_rate,'A2':a2_rate,'A3':a3_rate/360,'A4':a4_rate,'L1':-l1_rate/360,
                       'L2':-l2_rate/360,'L3':-l3_rate/360,'L4':-l4_rate,'L5':-l5_rate})
    cov_matrix = returns.cov()
    
    return np.sqrt(x.T@(cov/np.sqrt(360))@x) - np.sum(returns.sum()*x)

#### this function is used to check our final CFs which is basically

$x^T\mu$

In [7]:
def CFs(x):  # Defn Objective function

    returns = pd.DataFrame({'A1':a1_rate,'A2':a2_rate,'A3':a3_rate/360,'A4':a4_rate,'L1':-l1_rate/360,
                       'L2':-l2_rate/360,'L3':-l3_rate/360,'L4':-l4_rate,'L5':-l5_rate})
    cov_matrix = returns.cov()
    
    return - np.sum(returns.sum()*x)

#### below are the constraints

* constraints 1&2 is to define the total balance
* cons 3 : $NSFR >= 110\%$
* cons 4 : $\frac{TLAC}{RWA} >= 18\%$
* cons 5 : $LCR >= 120\%$
* cons 6 : get minimum amount of x4 to cover the extreme loss in CFs
* cons 7 : $x_3 >= 10\%$ of total balance
* cons 8 : $x_5+x_6+x_7 <= 60\%$ of total balance
* cons 9 : $x_6+x_7 = \eta*(x_5+x_6+x_7)$
* cons 10 : $x_6 = DRE*(x_6+x_7)$
* cons 11 : make sure all cashflows are positive which is not used in the case of allowing repo

In [8]:
def cons1(x):  # Constraints: Eq.(25) sum of the balance is 1550
    return x[0]+x[1]+x[2]+x[3]-Total_Bal

def cons2(x):
    return x[4]+x[5]+x[6]+x[7]+x[8]-Total_Bal

def cons3(x): # Eq.(21) NSFR >= 110%
    return sum(ASF*x[4:])/sum(RSF*x[:4]) - NSFR_ratio

def cons4(x): # Eq.(23) TLAC/RWA >= 18%
    return x[7]/sum(RWA*x[:4]) - TLAC_ratio

def cons5(x): # Eq.(22) LCR >= 120%
    
    x_cf1 = a1_rate*x[0]
    x_cf2 = a2_rate*x[1]
    x_cf3 = a3_rate*x[2]/360
    x_cf4 = a4_rate*x[3] 
    xa_cf = x_cf1 + x_cf2 + x_cf3 + x_cf4
    
    x_cf5 = (l1_rate*x[4])/360
    x_cf6 = (l2_rate*x[5])/360
    x_cf7 = (l3_rate*x[6])/360
    x_cf8 = (l4_rate*x[7])
    x_cf9 = (l5_rate*x[8])
    
    xl_cf = x_cf5 + x_cf6 + x_cf7 + x_cf8 + x_cf9
    
    xnet_cf = xa_cf - xl_cf
    
    HQLA_v = sum(HQLA * x[:4])

    TNCO = np.array([])

    for t in range(T):
        if t <= T-30:
            value = x[4] + x[5]*1/12 + x[6]*1/30 + sum(xnet_cf[t:t+30])
        else:
            value = x[4] + x[5]*1/12 + x[6]*1/30 + sum(xnet_cf[t:])
        TNCO = np.append(TNCO,value)
        
    return HQLA_v/TNCO - LCR_ratio

def cons6(x):  # Eq.(27) X4 to cover the loss
                      
    x_cf1 = a1_rate*x[0]
    x_cf2 = a2_rate*x[1]
    x_cf3 = a3_rate*x[2]/360
    x_cf4 = a4_rate*x[3] 
                      
    xa_cf = x_cf1 + x_cf2 + x_cf3 + x_cf4
    
    x_cf5 = (l1_rate*x[4])/360
    x_cf6 = (l2_rate*x[5])/360
    x_cf7 = (l3_rate*x[6])/360
    x_cf8 = (l4_rate*x[7])
    x_cf9 = (l5_rate*x[8])
    xl_cf = x_cf5 + x_cf6 + x_cf7 + x_cf8 + x_cf9
    
    xnet_cf = xa_cf - xl_cf
    xcum_cf = xnet_cf.cumsum()
    
    return x[3]*(1-Haircut)*(1-borrow_rate) + xcum_cf

def cons7(x): # Eq.(28)
    return x[2]-0.1*sum(x[:4])

def cons8(x): 
    return 0.6*Total_Bal - (x[4]+x[5]+x[6]) 

def cons9(x): 
    return x[5]+x[6] - Eta*(x[4]+x[5]+x[6])

def cons10(x):
    return x[5] - DRE*(x[5]+x[6])

def cons11(x):  # Eq.(27) Inflow > Outflow
                      
    x_cf1 = a1_rate*x[0]
    x_cf2 = a2_rate*x[1]
    x_cf3 = a3_rate*x[2]/360
    x_cf4 = a4_rate*x[3] 
                      
    xa_cf = x_cf1 + x_cf2 + x_cf3 + x_cf4
    
    x_cf5 = (l1_rate*x[4])/360
    x_cf6 = (l2_rate*x[5])/360
    x_cf7 = (l3_rate*x[6])/360
    x_cf8 = (l4_rate*x[7])
    x_cf9 = (l5_rate*x[8])
    xl_cf = x_cf5 + x_cf6 + x_cf7 + x_cf8 + x_cf9
    
    xnet_cf = xa_cf - xl_cf
    xcum_cf = xnet_cf.cumsum()
    
    return xcum_cf

#### define boundries of each $x_i$ as well as constraints
* 'eq': equation
* 'ineq': greater than or equal to zero

In [9]:
b = (0,Total_Bal) # Define bounds
bnds = (b, b, b, b, b, b, b, b, b)

con1 = {'type': 'eq', 'fun': cons1}
con2 = {'type': 'eq', 'fun': cons2}
con3 = {'type': 'ineq', 'fun': cons3}
con4 = {'type': 'ineq', 'fun': cons4}
con5 = {'type': 'ineq', 'fun': cons5}
con6 = {'type': 'ineq', 'fun': cons6}
con7 = {'type': 'ineq', 'fun': cons7}
con8 = {'type': 'ineq', 'fun': cons8}
con9 = {'type': 'eq', 'fun': cons9}
con10 = {'type': 'eq', 'fun': cons10}

cons = ([con1,con2,con3,con4,con5,con6,con7,con8,con9,con10])
#cons = ([con1,con2,con3,con4,con5,con6,con7,con8,con9,con10])

In [10]:
x = np.array([375000,375000,375000,375000,300000,300000,300000,300000,300000])

##### optimization function, run this multiple times untile it converges

In [15]:
solution = minimize(obj_mean_var,x,bounds=bnds,constraints=cons,method='SLSQP')

x = solution.x

#### create the repo table including columns:
* NCNC : all the possible negative cashflows
* Repo Amt : Amount needed to repo, shift NCNC oneday ago plus the haircut
* Interest : interests generated from repo

In [16]:
neg_cfs = cons11(x)[cons11(x)<0]

repo_df = pd.DataFrame(index=np.array(['t'+'{}'.format(i) for i in range(len(neg_cfs)+1)])) # create repo table
repo_df['NCNC'] = [0]+cons11(x)[cons11(x)<0].tolist()
repo_df['Repo Amt'] = (repo_df['NCNC']/(1-Haircut)).shift(-1)
repo_df['Interest'] = repo_df['Repo Amt'] * borrow_rate

#### Final results

In [17]:
print('Weights:', np.round(x,2))
print('PnL without repo:',-CFs(x))
print('Final PnL:',-CFs(x) + repo_df['Interest'].sum())
print('Obj mean-variance:',obj_mean_var(x))

Weights: [5.5912606e+05 7.9013406e+05 1.5000000e+05 7.3988000e+02 3.3300000e+05
 1.0206000e+05 4.6494000e+05 1.5044777e+05 4.4955223e+05]
PnL without repo: 33858.87216961898
Final PnL: 33802.625385411906
Obj mean-variance: -11372.008006994842


In [18]:
repo_df

,NCNC,Repo Amt,Interest
t0,0.000000,-9.255387,-0.015734
t1,-9.070279,-18.510774,-0.031468
t2,-18.140558,-27.766161,-0.047202
t3,-27.210838,-37.021548,-0.062937
t4,-36.281117,-46.147270,-0.078450
...,...,...,...
t85,-690.201476,-712.860840,-1.211863
t86,-698.603623,-721.441159,-1.226450
t87,-707.012336,-730.028178,-1.241048
t88,-715.427614,-738.621897,-1.255657
